Import libraries

In [22]:
# Install necessary packages
!pip install pandas scikit-learn tensorflow



For training validation and testing


In [13]:
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math

# Load the dataset
dataset = pd.read_csv('/content/dataset.csv')

# Extract features and target variable
X = dataset[['Wind Speed (m/s)', 'Theoretical_Power_Curve (KWh)', 'Wind Direction (°)']]
y = dataset['LV ActivePower (kW)']

# Normalize features
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

# Split the data into train, validate, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X_normalized, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1), kernel_regularizer=l1(0.01)))
model.add(Dense(1))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['mae'])

# Reshape data for LSTM (add time dimension)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model on the test set
loss, mae = model.evaluate(X_test, y_test)
print(f'Test Mean Absolute Error: {mae}')

# Assuming 'y_test_last_20_pred' is a 2D array
# Inverse transform the normalized predictions to get actual ActivePower values
scaler = MinMaxScaler()
y = y.values.reshape(-1, 1)
scaler.fit(y)

# Get last 1440 rows from test set
last_1440 = X_test.shape[0] - 1440

X_test_last_1440 = X_test[last_1440:]
y_test_last_1440 = y_test[last_1440:]

# Make predictions
y_pred_last_1440 = model.predict(X_test_last_1440)

# Inverse transform predictions
y_pred_last_1440 = scaler.inverse_transform(y_pred_last_1440).flatten()

# Calculate RMSE
rmse = math.sqrt(mean_squared_error(y_test_last_1440, y_pred_last_1440))

# Create DataFrame
result_df = pd.DataFrame({
    'Actual': y_test_last_1440,
    'Predicted': y_pred_last_1440,
    'RMSE': rmse
})

print(result_df)

# Create rows to write
rows = [
    [actual, predicted, rmse]
    for actual, predicted in zip(y_test_last_1440, y_pred_last_1440)
]


# Open csv file for writing
with open('predictions.csv', 'w') as f:

    # Create csv writer
    writer = csv.writer(f)

    # Write column headers
    writer.writerow(['Actual', 'Predicted', 'RMSE'])

    # Write each row
    writer.writerows(rows)

print('CSV saved successfully!')



Epoch 1/50
948/948 [==============================] - 3s 3ms/step - loss: 386928.2812 - mae: 358.5123 - val_loss: 159048.9531 - val_mae: 205.7483
Epoch 2/50
948/948 [==============================] - 2s 2ms/step - loss: 169052.9688 - mae: 197.5963 - val_loss: 162422.6719 - val_mae: 157.3386
Epoch 3/50
948/948 [==============================] - 3s 3ms/step - loss: 163661.4688 - mae: 180.8880 - val_loss: 150917.0781 - val_mae: 184.7164
Epoch 4/50
948/948 [==============================] - 3s 3ms/step - loss: 161693.3281 - mae: 174.8872 - val_loss: 147493.2812 - val_mae: 164.4979
Epoch 5/50
948/948 [==============================] - 2s 2ms/step - loss: 160761.6875 - mae: 174.0739 - val_loss: 150294.9844 - val_mae: 183.8889
Epoch 6/50
948/948 [==============================] - 3s 3ms/step - loss: 160160.4531 - mae: 172.5674 - val_loss: 150268.3594 - val_mae: 171.1640
Epoch 7/50
948/948 [==============================] - 3s 3ms/step - loss: 159240.1406 - mae: 171.2404 - val_loss: 150787.750

Future Prediction - 24 hrs


In [21]:
import numpy as np

# Train model only on training data
model.fit(X_train, y_train, epochs=50)

# No of features
no_features = X_train.shape[1]

# Create dummy 2D input
next_1440 = pd.DataFrame(np.zeros((1440, no_features)))

# Make predictions
y_pred_next_1440 = model.predict(next_1440)

# Inverse transform predictions
y_pred_next_1440 = scaler.inverse_transform(y_pred_next_1440)

# RMSE will be Nan as we don't have actual values
rmse = [np.nan]*len(y_pred_next_1440)

# Prepare rows for writing to CSV
rows = [
    [np.nan, pred, rmse_val]
    for pred, rmse_val in zip(y_pred_next_1440, rmse)
]

# Write predictions to CSV
with open('future_predictions.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['Actual','Predicted','RMSE'])
    writer.writerows(rows)

print(rmse)

print('Future predictions saved to CSV')

Epoch 1/50
948/948 [==============================] - 2s 2ms/step - loss: 152794.4062 - mae: 162.0071
Epoch 2/50
948/948 [==============================] - 2s 2ms/step - loss: 152913.3125 - mae: 162.5797
Epoch 3/50
948/948 [==============================] - 2s 2ms/step - loss: 152868.5938 - mae: 162.3024
Epoch 4/50
948/948 [==============================] - 2s 2ms/step - loss: 152964.4375 - mae: 162.1060
Epoch 5/50
948/948 [==============================] - 2s 2ms/step - loss: 152757.5625 - mae: 162.7124
Epoch 6/50
948/948 [==============================] - 2s 2ms/step - loss: 152963.2500 - mae: 161.8400
Epoch 7/50
948/948 [==============================] - 2s 2ms/step - loss: 153008.6094 - mae: 161.7057
Epoch 8/50
948/948 [==============================] - 2s 2ms/step - loss: 152908.5000 - mae: 162.6572
Epoch 9/50
948/948 [==============================] - 2s 2ms/step - loss: 152559.6250 - mae: 161.3618
Epoch 10/50
948/948 [==============================] - 2s 2ms/step - loss: 152996.

Please use following link to see output files and resources - https://drive.google.com/drive/folders/1P3PMPAP9C5IkDpmq3iyoC6WPdJL2SYMV?usp=sharing